# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
3/3 [==============================] - 0s 122ms/step - loss: 2.3859 - accuracy: 0.0294 - val_loss: 2.3010 - val_accuracy: 0.1771
Epoch 2/5
3/3 [==============================] - 0s 118ms/step - loss: 2.2946 - accuracy: 0.1146 - val_loss: 2.3007 - val_accuracy: 0.1771
Epoch 3/5
3/3 [==============================] - 0s 117ms/step - loss: 2.3020 - accuracy: 0.1029 - val_loss: 2.3089 - val_accuracy: 0.1042
Epoch 4/5
3/3 [==============================] - 0s 86ms/step - loss: 2.2648 - accuracy: 0.1471 - val_loss: 2.3282 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 0s 82ms/step - loss: 2.3121 - accuracy: 0.0588 - val_loss: 2.3774 - val_accuracy: 0.0729


# Load Checkpoint

In [ ]:
model.load_weights('checkpoints')

# Saving Model

In [16]:
save_path = 'mymodel.h5'

In [17]:
model.save(save_path, include_optimizer=True)

In [18]:
model_ = tf.keras.models.load_model('mymodel.h5')

# Saving Model - 2

In [19]:
model.save_weights('model_weights.h5')

In [20]:
with open('model_architecture.json','w') as f:
    f.write(model.to_json())

# model.h5 들여다보기

In [21]:
import h5py

In [22]:
model_file = h5py.File('mymodel.h5', 'r+')

In [23]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [24]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [25]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [26]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [27]:
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

array([[[[ 1.29264649e-02, -7.43630752e-02, -1.15566693e-01,
           1.15833052e-01, -1.01491444e-01,  5.43681085e-02,
          -1.22614704e-01, -1.06557727e-01,  6.41910136e-02,
           5.55084348e-02, -1.34538695e-01, -3.74212712e-02,
           1.83644164e-02,  8.79505724e-02,  3.56111601e-02,
           5.53667210e-02, -4.82676737e-03,  2.04892107e-03,
          -8.92910734e-02,  1.36247113e-01, -5.26835024e-02,
           1.19112907e-02,  4.24701832e-02,  5.10574728e-02,
          -9.26299021e-02, -2.83396617e-02,  1.74181145e-02,
          -1.38688296e-01,  3.59615907e-02, -1.12014927e-01,
           5.48468642e-02, -1.04599237e-01],
         [ 1.00609608e-01,  6.19837381e-02,  3.40131000e-02,
           5.79758435e-02,  1.93704944e-02,  2.95441896e-02,
           1.58071723e-02, -2.45018303e-02, -2.92502996e-02,
           5.15227532e-03,  6.97738677e-02, -1.34020731e-01,
           1.05090886e-01, -1.09662063e-01,  8.80536959e-02,
           3.67647745e-02,  1.25078529e-